In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import requests
import json

import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from IPython.core.display import HTML, display

C:\Users\tomas\AppData\Local\Temp\ipykernel_1908\1759760073.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


#### Lecture 4a - JSONs and APIs
by Vítek Macháček
October 25, 2022

### Contents

* Standardized data representation
* JSON (+ XMLs)
* Introduction to Requests (GET vs. POST) and APIs


### Goals:
    
* work with data  online/real-time data
* acquisition, processing - > results
* Today introduction and next week a practical example

## Microservice architecture

* Foundation of modern software architecture
* Do one thing and do it well.

![Microservice architecture schema](./img/microservices.png "Microservice Architecture")


## Date exchange formats - JSON, XML

`Language of the internet`

* You can send/receive a message with (almost) any service
* we need system agnostic data format 
* is ediatable in basic editors
* More complex than simple tables
* Highly structured - if you dont follow the rules, you are out
* Both sides need to understand the structure
* only data. It does not do anything - no code to be run
* distributed as text/string (to be precise as `bytes` literals) 
* parsed to objects - easy to work with straight away
* Can be persisted as special files, or some data streams from APIs. 
* Human readable
* Hierarchical
* Can be fetched using standard web APIs

### Purpose

1. Communication 
    * All imaginable communication channels
    * Applications

2. Storing
    * self-descriptive
    * human readable
    * also in DBs - SQL, MongoDB etc.

3. Standardization
    * predictability
    * cooperation
    * spillovers from standardization

### Dimensionality problem

* rich information comes at costs of data complexity 
* to interrelate information, you need to high dimensionality (or A LOT of columns)
* Strongly object-oriented


### 1D:
* logs

### 2D
* tabular data (like pandas DFs)
* SQL

### 3+D:
#### XML (and HTML)
* eXtensible Markup Language is a software- and hardware-independent tool for storing and transporting data.
* Officialy defined at 1998, but its roots are even older.
* XML was designed to carry data - with focus on what data is
* HTML was designed to display data - with focus on what data should look like displayed
* XML tags are not predefined like HTML tags are
* more verbose than JSON
* can have comments !actually a really cool in useful feature!
* used historically as a transaction format in many areas: 
    * Scientific measurements
    * News information
    * Wheather measurements
    * Financial transactions
* Necessary to use XML parser - for example `BeautifulSoup` or `xmltree`
* doc`x`, xls`x`, etc. stands for xml


### JSON
* JavaScript Object Notation
* REST APIs return JSONs
* often *.json* files
* but also used in the web etc.
* supports standard datatypes - strings, integers, floats, lists
* No comments
* More compact, less verbose
* No closing tags
* Used EVERYWHERE, BUT [NOT LICENSED FOR EVIL](https://www.json.org/license.html). If you want to do evil stuff, use XML instead.
* Native in JavaScript and close to native in Python (dictionary)
* Jupyter Notebooks

# JSON

* JSON is similar to combination of `dictionaries` (`object` in JSON-terms)  and `lists` (`arrays`) in Python

In [2]:
teachers = [
    {'name':'Jozef Baruník','titles':['doc.','PhDr.','Ph.D.','Bc.','Mgr.'],'ID':1234,'courses':['JEM005','JEM116','JEM059','JEM061']},
    {'name':'Martin Hronec','titles':['Bc.','Mgr.'],'ID':3421,'courses':['JEM005','JEM207']},
]

courses = {
    "JEM005":{'name':'Advanced Econometrics','ECTS':6,'teachers':[3421,1234]},
    'JEM207':{'name':'Data Processing in Python','ECTS':5,'teachers':[3421]},
    'JEM116':{'name':'Applied Econometrics','ECTS':6,'teachers':[1234]},
    'JEM059':{'name':'Quantitative Finance I.','ECTS':6,'teachers':[1234,5678]},
    'JEM061':{'name':'Quantitative Finance II.','ECTS':6,'teachers':[1234,5678]}
}
jsondata = {'teachers':teachers,'courses':courses}
jsondata

{'teachers': [{'name': 'Jozef Baruník',
   'titles': ['doc.', 'PhDr.', 'Ph.D.', 'Bc.', 'Mgr.'],
   'ID': 1234,
   'courses': ['JEM005', 'JEM116', 'JEM059', 'JEM061']},
  {'name': 'Martin Hronec',
   'titles': ['Bc.', 'Mgr.'],
   'ID': 3421,
   'courses': ['JEM005', 'JEM207']}],
 'courses': {'JEM005': {'name': 'Advanced Econometrics',
   'ECTS': 6,
   'teachers': [3421, 1234]},
  'JEM207': {'name': 'Data Processing in Python',
   'ECTS': 5,
   'teachers': [3421]},
  'JEM116': {'name': 'Applied Econometrics', 'ECTS': 6, 'teachers': [1234]},
  'JEM059': {'name': 'Quantitative Finance I.',
   'ECTS': 6,
   'teachers': [1234, 5678]},
  'JEM061': {'name': 'Quantitative Finance II.',
   'ECTS': 6,
   'teachers': [1234, 5678]}}}

is this a valid JSON?

https://jsonformatter.curiousconcept.com/

In [3]:
js = json.dumps(
    jsondata, indent=4, ensure_ascii = False
) #json formatted string!

print(js)

{
    "teachers": [
        {
            "name": "Jozef Baruník",
            "titles": [
                "doc.",
                "PhDr.",
                "Ph.D.",
                "Bc.",
                "Mgr."
            ],
            "ID": 1234,
            "courses": [
                "JEM005",
                "JEM116",
                "JEM059",
                "JEM061"
            ]
        },
        {
            "name": "Martin Hronec",
            "titles": [
                "Bc.",
                "Mgr."
            ],
            "ID": 3421,
            "courses": [
                "JEM005",
                "JEM207"
            ]
        }
    ],
    "courses": {
        "JEM005": {
            "name": "Advanced Econometrics",
            "ECTS": 6,
            "teachers": [
                3421,
                1234
            ]
        },
        "JEM207": {
            "name": "Data Processing in Python",
            "ECTS": 5,
            "teachers": [
                3

# Reading data using `requests` library

* API = Application Programming Interface
* more specifically: http based APIs

### When to use?
* whenever more applications need to communicate - 
    * DB speaks to app
    * accounting system communicates with inventory system
    * Google Maps need to get info about local public transport
    * ML-based BitCoin price prediction to be used to facilitate automatic trading
    *
* user-friendly interface for complicated tasks - DEEP AI, Google Maps
* Data - Golemio, OpenStreetMaps

## HTTP request

* A most standard webserver communication channel around
* `Client` asks/requests questions - **requests**
* `Server` replies/serve answers - **responses**

### HTTP request structure:
* URL
    * domain
    * route
    * parameters
* Request Type - GET, POST, PUT, DELETE
* Request Header
    * authentication
    * cookies
    * other metadata
* Outcoming data (will see below)
   
### HTTP response structure
* Header 
    * cookies
    * other metadata - responding server, dates, 
* Status Code:
    * 200 - success
    * 404 - resource does not exist
    * 500 - the server failed during processing your request
* Content
    * text - JSON, HTML etc.
    * file

### API types
1) REST API - use HTTP request and returns JSON
2) SOAP API - use HTTP request and returns XML
3) Website - use HTTP request and returns set of HTML, JavaScript, CSS and other files


### GET request
* fast
* public
* data flow only one direction
* parameters via request adress

### POST request
* slow
* private
* both sides can send data

### The simplest request

In [4]:
import requests

In [5]:
r = requests.get('https://www.google.com/')
print(r.text)

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="cs"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="f3AB83I-_nTQff9FEGuCUQ">(function(){window.google={kEI:'OKUQZP2RBeK68gKNmoBo',kEXPI:'0,1359409,6059,206,4804,2316,383,246,5,1129120,1197702,869,379920,16111,28687,22430,1362,12316,17583,4998,13228,3847,38444,2872,2891,561,3578,7615,606,60690,2614,13142,3,346,230,6460,14123,4,1528,2304,42125,13660,4437,9358,7428,5830,2527,4094,7596,1,42154,2,14022,25739,6700,31121,4568,6256,23421,1252,5835,12200,2767,4333,7484,25076,2006,5895,2260,7381,15969,874,3087,16546,7,1922,5784,3995,12414,8976,389,1114,13261,6305,2007,18191,11814,8419,13250,6956,1622,1779,669,4307,3940,10951,4069,280,4135,988,3030,426,2608,3072,4,1411,890,2740,3897,768,523,780,501,567,6983,679,1152,1086,5,269,1380,108,67,452,608,885,12593,933,1106,348

In [6]:
r = requests.get('https://www.google.com/')
display(HTML(r.text))

,,Rozšířené vyhledávání


## "Real-world" APIs

### Sreality

* surprisingly no need for authentication
https://www.sreality.cz/hledani/prodej/byty/praha

In [7]:
r = requests.get('https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&locality_region_id=10&per_page=20&tms=1678732084920')
r.text

'{"meta_description": "5858 realit v nab\\u00eddce prodej byt\\u016f Praha. Vyberte si novou nemovitost na sreality.cz s hled\\u00e1n\\u00edm na map\\u011b a velk\\u00fdmi n\\u00e1hledy fotografi\\u00ed nab\\u00edzen\\u00fdch byt\\u016f.", "result_size": 5858, "_embedded": {"estates": [{"labelsReleased": [["new_building", "parking_lots"], ["post_office", "drugstore"]], "has_panorama": 0, "labels": ["Novostavba", "Parkov\\u00e1n\\u00ed", "Po\\u0161ta 8 min. p\\u011b\\u0161ky", "L\\u00e9k\\u00e1rna 2 min. p\\u011b\\u0161ky"], "is_auction": false, "labelsAll": [["new_building", "personal", "balcony", "cellar", "elevator", "parking_lots", "garage"], ["playground", "vet", "small_shop", "candy_shop", "tavern", "theater", "sightseeing", "movies", "bus_public_transport", "school", "atm", "sports", "shop", "metro", "drugstore", "tram", "kindergarten", "train", "post_office", "restaurant", "medic"]], "seo": {"category_main_cb": 1, "category_sub_cb": 2, "category_type_cb": 1, "locality": "praha-z

In [8]:
type(r.json())

dict

In [9]:
r = requests.get('https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&locality_region_id=10')
r.text

'{"meta_description": "5858 realit v nab\\u00eddce prodej byt\\u016f Praha. Vyberte si novou nemovitost na sreality.cz s hled\\u00e1n\\u00edm na map\\u011b a velk\\u00fdmi n\\u00e1hledy fotografi\\u00ed nab\\u00edzen\\u00fdch byt\\u016f.", "result_size": 5858, "_embedded": {"estates": [{"labelsReleased": [[], ["post_office"]], "has_panorama": 0, "labels": ["Po\\u0161ta 6 min. p\\u011b\\u0161ky"], "is_auction": false, "labelsAll": [["personal", "brick", "cellar", "partly_furnished"], ["playground", "natural_attraction", "small_shop", "candy_shop", "vet", "tavern", "theater", "sightseeing", "movies", "post_office", "atm", "drugstore", "shop", "restaurant", "sports", "metro", "tram", "school", "bus_public_transport", "kindergarten", "train", "medic"]], "seo": {"category_main_cb": 1, "category_sub_cb": 6, "category_type_cb": 1, "locality": "praha-vinohrady-borivojova"}, "exclusively_at_rk": 0, "category": 1, "has_floor_plan": 0, "_embedded": {"favourite": {"is_favourite": false, "_links": 

In [10]:
d = r.json()

In [11]:
d.keys()

dict_keys(['meta_description', 'result_size', '_embedded', 'filterLabels', 'title', 'filter', '_links', 'locality', 'locality_dativ', 'logged_in', 'per_page', 'category_instrumental', 'page', 'filterLabels2'])

In [12]:
pd.json_normalize(d['_embedded']['estates'])

,labelsReleased,has_panorama,labels,is_auction,labelsAll,exclusively_at_rk,category,has_floor_plan,paid_logo,locality,...,price_czk.unit,price_czk.name,_links.dynamicDown,_links.dynamicUp,_links.iterator.href,_links.self.href,_links.images,_links.image_middle2,gps.lat,gps.lon
0,"[[], [post_office]]",0,[Pošta 6 min. pěšky],False,"[[personal, brick, cellar, partly_furnished], ...",0,1,0,1,Praha 3 - Vinohrady,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,/cs/v2/estate-iterator/0?category_main_cb=1&su...,/cs/v2/estates/3751355468?region_tip=2443542,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,50.070537,14.463589
1,"[[new_building, parking_lots, garage], []]",0,"[Novostavba, Parkování, Garáž]",False,"[[new_building, personal, parking_lots, garage...",0,1,0,1,Praha 4 - Újezd u Průhonic,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,/cs/v2/estate-iterator/1?category_main_cb=1&su...,/cs/v2/estates/419370572,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,50.002052,14.554065
2,"[[in_construction, terrace, garage], []]",0,"[Ve výstavbě, Terasa, Garáž]",False,"[[in_construction, personal, terrace, cellar, ...",1,1,1,1,Praha 8 - Libeň,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QM_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QM_...,/cs/v2/estate-iterator/2?category_main_cb=1&su...,/cs/v2/estates/3873489996,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QM_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QM_...,50.105552,14.486164
3,"[[new_building, loggia, garage], [shop]]",0,"[Novostavba, Lodžie, Garáž, Obchod 7 min. pěšky]",False,"[[new_building, personal, loggia, brick, eleva...",1,1,0,0,Praha 5 - Stodůlky,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QQ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QQ_...,/cs/v2/estate-iterator/3?category_main_cb=1&su...,/cs/v2/estates/3640744284,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QQ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QQ_...,50.034484,14.321338
4,"[[], []]",0,[],False,"[[personal, balcony, brick, cellar, partly_fur...",0,1,1,0,Praha 3 - Žižkov,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,/cs/v2/estate-iterator/4?category_main_cb=1&su...,/cs/v2/estates/1178977356,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,50.083469,14.488569
5,"[[new_building, parking_lots, garage], []]",0,"[Novostavba, Parkování, Garáž]",False,"[[new_building, personal, balcony, brick, cell...",1,1,0,1,Praha 10 - Horní Měcholupy,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,/cs/v2/estate-iterator/5?category_main_cb=1&su...,/cs/v2/estates/326162252,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,50.034145,14.558661
6,"[[], [medic]]",0,[Lékař 4 min. pěšky],False,"[[personal, brick, cellar, partly_furnished], ...",0,1,1,0,Praha 5 - Košíře,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QR_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QR_...,/cs/v2/estate-iterator/6?category_main_cb=1&su...,/cs/v2/estates/1346032716,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QR_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QR_...,50.059459,14.370592
7,"[[not_furnished], []]",0,[Nevybavený],False,"[[personal, brick, not_furnished], [playground...",0,1,1,0,Praha 5 - Smíchov,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QK_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QK_...,/cs/v2/estate-iterator/7?category_main_cb=1&su...,/cs/v2/estates/3600069708,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QK_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QK_...,50.059356,14.357041
8,"[[after_reconst

In [13]:
pd.json_normalize(d['_embedded']['estates'])

,labelsReleased,has_panorama,labels,is_auction,labelsAll,exclusively_at_rk,category,has_floor_plan,paid_logo,locality,...,price_czk.unit,price_czk.name,_links.dynamicDown,_links.dynamicUp,_links.iterator.href,_links.self.href,_links.images,_links.image_middle2,gps.lat,gps.lon
0,"[[], [post_office]]",0,[Pošta 6 min. pěšky],False,"[[personal, brick, cellar, partly_furnished], ...",0,1,0,1,Praha 3 - Vinohrady,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,/cs/v2/estate-iterator/0?category_main_cb=1&su...,/cs/v2/estates/3751355468?region_tip=2443542,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,50.070537,14.463589
1,"[[new_building, parking_lots, garage], []]",0,"[Novostavba, Parkování, Garáž]",False,"[[new_building, personal, parking_lots, garage...",0,1,0,1,Praha 4 - Újezd u Průhonic,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,/cs/v2/estate-iterator/1?category_main_cb=1&su...,/cs/v2/estates/419370572,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,50.002052,14.554065
2,"[[in_construction, terrace, garage], []]",0,"[Ve výstavbě, Terasa, Garáž]",False,"[[in_construction, personal, terrace, cellar, ...",1,1,1,1,Praha 8 - Libeň,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QM_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QM_...,/cs/v2/estate-iterator/2?category_main_cb=1&su...,/cs/v2/estates/3873489996,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QM_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QM_...,50.105552,14.486164
3,"[[new_building, loggia, garage], [shop]]",0,"[Novostavba, Lodžie, Garáž, Obchod 7 min. pěšky]",False,"[[new_building, personal, loggia, brick, eleva...",1,1,0,0,Praha 5 - Stodůlky,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QQ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QQ_...,/cs/v2/estate-iterator/3?category_main_cb=1&su...,/cs/v2/estates/3640744284,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QQ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QQ_...,50.034484,14.321338
4,"[[], []]",0,[],False,"[[personal, balcony, brick, cellar, partly_fur...",0,1,1,0,Praha 3 - Žižkov,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,/cs/v2/estate-iterator/4?category_main_cb=1&su...,/cs/v2/estates/1178977356,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QJ_...,50.083469,14.488569
5,"[[new_building, parking_lots, garage], []]",0,"[Novostavba, Parkování, Garáž]",False,"[[new_building, personal, balcony, brick, cell...",1,1,0,1,Praha 10 - Horní Měcholupy,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,/cs/v2/estate-iterator/5?category_main_cb=1&su...,/cs/v2/estates/326162252,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QO_...,50.034145,14.558661
6,"[[], [medic]]",0,[Lékař 4 min. pěšky],False,"[[personal, brick, cellar, partly_furnished], ...",0,1,1,0,Praha 5 - Košíře,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QR_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QR_...,/cs/v2/estate-iterator/6?category_main_cb=1&su...,/cs/v2/estates/1346032716,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QR_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QR_...,50.059459,14.370592
7,"[[not_furnished], []]",0,[Nevybavený],False,"[[personal, brick, not_furnished], [playground...",0,1,1,0,Praha 5 - Smíchov,...,,Celková cena,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QK_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QK_...,/cs/v2/estate-iterator/7?category_main_cb=1&su...,/cs/v2/estates/3600069708,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QK_...,[{'href': 'https://d18-a.sdn.cz/d_18/c_img_QK_...,50.059356,14.357041
8,"[[after_reconst

### World Bank

#### Exploratory request

In [14]:
d = requests.get('http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json&per_page=100').json()

In [15]:
d[0]

{'page': 1,
 'pages': 165,
 'per_page': 100,
 'total': 16492,
 'sourceid': '2',
 'sourcename': 'World Development Indicators',
 'lastupdated': '2023-03-01'}

#### Look at the data in the dataframe

In [16]:
pd.json_normalize(d[1])

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2021,702976832,,,0,SP.POP.TOTL,"Population, total",ZH,Africa Eastern and Southern
1,AFE,2020,685112705,,,0,SP.POP.TOTL,"Population, total",ZH,Africa Eastern and Southern
2,AFE,2019,667242712,,,0,SP.POP.TOTL,"Population, total",ZH,Africa Eastern and Southern
3,AFE,2018,649756874,,,0,SP.POP.TOTL,"Population, total",ZH,Africa Eastern and Southern
4,AFE,2017,632746296,,,0,SP.POP.TOTL,"Population, total",ZH,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...,...
95,AFW,1988,195969722,,,0,SP.POP.TOTL,"Population, total",ZI,Africa Western and Central
96,AFW,1987,190759952,,,0,SP.POP.TOTL,"Population, total",ZI,Africa Western and Central
97,AFW,1986,185720244,,,0,SP.POP.TOTL,"Population, total",ZI,Africa Western and Central
98,AFW,1985,180817312,,,0,SP.POP.TOTL,"Population, total",ZI,Africa Western and Central


### More advanced example: Paging

#### Return to Python basics 1: Errors and exceptions in Python

In [17]:
for i in range(-5,5):
    print(5/i)

-1.0
-1.25
-1.6666666666666667
-2.5
-5.0


ZeroDivisionError: division by zero

In [ ]:
for i in range(-5,5):
    try:
        print(1/i)
    except:
        print(f'dividing with {i} raised an error. Are you sure your input was correct?')

-0.2
-0.25
-0.3333333333333333
-0.5
-1.0
dividing with 0 raised an error. Are you sure your input was correct?
1.0
0.5
0.3333333333333333
0.25


Return to Python basics 2: Formatting strings

In [ ]:
my_name = 'Vítek'

f'Hello {my_name}!'

In [ ]:
string_template = 'Today {teachers_name} is teaching and he is in the {teachers_mood} mood'

string_template.format(
    teachers_name='Vítek',
    teachers_mood='good'
)

Sending API requests is always risky - you do not control the other side of the transaction

Try listing first ten pages of results in the request

Always check if everything goes fine by checking the request status code

In [ ]:
l = []
url_template = 'http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json&page={page_num}'

for i in range(1,10):
    r = requests.get(url_template.format(page_num=i))    
    if r.ok: #r.status_code == 200 would also work!
        l.append(r.json())

OK, but you still have imply a strong confidence on the other side, try and except is more certain

In [ ]:
l = []
url_template = 'http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json&page={page_num}'

for i in range(1,10):
    url = url_template.format(page_num=i)
    try:
        r = requests.get(url)    
        if r.ok: #r.status_code == 200 would also work!
            l.append(r.json())
    except:
        print(f'At least I want to know that something went wrong and when. Url: {url}')

OMG, this looks a bit messy. I would consider a writing function to increase clarity

In [ ]:
def request_worldbank(url):
    try:
        r = requests.get(url)    
        if r.ok: #r.status_code == 200 would also work!
            return r.json()
        else:
            print(f'The url request on {url} not succesful. Status code: {r.status_code}. Message: {r.message}')
    except:
        print(f'At least I want to know that something went wrong and when. Url: {url}')

l = []
url_template = 'http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json&page={page_num}'
for i in range(1,10):
    l.append(request_worldbank(url_template.format(page_num=i)))

In [ ]:
l = [request_worldbank(url_template.format(page_num=i)) for i in range(1,10)]

pd.concat([pd.json_normalize(output_json[1]) for output_json in l])

In [ ]:
def download_worldbank(indicator):
    url_template = 'http://api.worldbank.org/v2/country/all/indicator/{indicator}?format=json&page={page}&per_page=500'
    first_request = requests.get(url_template.format(indicator=indicator,page=1)).json()
        
    pages = first_request[0]['pages']
    
    def single_worldbank_request(url):
        try:
            r = requests.get(url)
            if r.ok:
                return pd.json_normalize(r.json()[1])
        except Exception as e:
            print(f'Could not parse an URL {url}. Read the message: {e.msg}')
    
    first_data = pd.json_normalize(first_request[1])
    
    l = [single_worldbank_request(url_template.format(indicator=indicator,page=page)) for page in range(2,pages+1)]

    return pd.concat([first_data] + l).set_index(['countryiso3code','date']).value
        
population = download_worldbank('SP.POP.TOTL')
population

### Eurostat

In [ ]:
from io import StringIO
r_gdp = requests.get('https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/NAMA_10_GDP?format=SDMX-CSV')
gdp = pd.read_csv(StringIO(r_gdp.text))
gdp

In [ ]:
gdp.unit.unique()

In [ ]:
gdp.na_item.unique()

In [ ]:
geo

In [ ]:
gdp['freq'].unique()

### Twitter

### Scopus

In [ ]:
from secret import SCOPUS_API_KEY
r = requests.get('https://api.elsevier.com/content/search/scopus?query=AUTH(baruník, j.)  ',headers={'Accept':'application/json','X-ELS-APIKey': SCOPUS_API_KEY})

pd.json_normalize(r.json()['search-results']['entry'])

### XML or even HTML data

In [ ]:
response = requests.get('https://en.wikipedia.org/wiki/Charles_University')
soup = BeautifulSoup(response.text)
div = soup.find('div',{'id':'mw-content-text'}) #  #mw-content-text > div > p:nth-child(10)texts)
article = ' '.join([p.text for p in div.find_all('p')])
print(article)

# Bonus example:

<img src="http://ies.fsv.cuni.cz/default/file/get/id/31996" height="500" width="300">

Will not work without authentication.

* You will need IAM account for Amazon Web Service 
* For that you can create `AWS_ACCESS_KEY` and `AWS_SECRET_KEY`. See here: https://aws.amazon.com/premiumsupport/knowledge-center/create-access-key/
* create `secret.py` file and put `AWS_ACCESS_KEY` and `AWS_SECRET_KEY`. Follow the template of `secret-example.py`

In [ ]:
!pip install boto3

In [ ]:
import boto3

In [ ]:
from secret import AWS_ACCESS_KEY, AWS_SECRET_KEY

client=boto3.client('rekognition', 
                    region_name='us-west-2',
                    aws_access_key_id=AWS_ACCESS_KEY,
                    aws_secret_access_key=AWS_SECRET_KEY
)

with open('./img/iespic.jpeg','rb') as f:
    response = client.recognize_celebrities(Image={'Bytes': f.read()})
pd.DataFrame(response['UnrecognizedFaces'][0]['Emotions']).set_index('Type').Confidence.plot.bar()

In [ ]:
response